In [141]:
#### Imports ####
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.pyplot import imshow, show, subplot,title,axis
from matplotlib.patches import Circle
from skimage.io import imread
import skimage.util
import skimage as sk
from timeit import timeit
from skimage.feature import corner_peaks,peak_local_max
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import os
from skimage import exposure, filters, morphology
from cellpose import models, io, denoise, utils
from cellpose.io import imread, logger_setup
import pandas as pd
from skimage.measure import regionprops, label, regionprops_table, shannon_entropy, euler_number
from skimage.feature import blob_log, graycomatrix, graycoprops
from skimage.exposure import rescale_intensity
from skimage import transform
from skimage.draw import polygon_perimeter
from scipy.ndimage import binary_erosion
import mahotas as mh


from Functions_George import *

In [142]:
# import os

# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  
# os.environ["OMP_NUM_THREADS"] = "1"  

paths = [r"C:\Users\George\Desktop\Final_Project\Images\Control", r"C:\Users\George\Desktop\Final_Project\Images\Penetramax"]

In [143]:
io.logger_setup()

all_props = []
Nuclei = []

for path in paths:
    Images = load_images_from_folder(path)

    #Model without and with denoising
    # model = models.Cellpose(model_type='nuclei', gpu=True)
    model = denoise.CellposeDenoiseModel(gpu=True, model_type="nuclei", restore_type='denoise_nuclei')
    
    for i in range(len(Images)):
        #1.Isolating the red channel for each Image
        Red_channel = np.copy(Images[i][:, :, 0])

        #2 Apply Adaptive Histogram equalization (play with kernal size and clip_limit), works better here
        Red_channel = skimage.exposure.equalize_adapthist(Red_channel, kernel_size=30, clip_limit=0.01, nbins=512)

        #3.Segmentation
        masks, flows, styles, diams = model.eval(Red_channel, diameter=60, channels=[0, 0])
    
        # fig, axes = plt.subplots(1, 3, figsize=(20, 6))
        # ax = axes.ravel()

        # ax[0].imshow(Images[i])
        # ax[0].set_title('Original Image')
        # ax[1].imshow(Red_channel, cmap='gray')
        # ax[1].set_title('Brightened Image')
        # ax[2].imshow(Images[i])
        # ax[2].imshow(masks, alpha = 0.9*(masks>0.01), cmap = custom_colormap())
        # ax[2].set_title('Segmented Image')

        # for a in ax:
        #     a.axis('off')

        # plt.suptitle(f'Image {i + 1}', x = 0.5, y = 0.97, fontsize = 25)
        # plt.tight_layout()
        # plt.show()

        props = regionprops(masks, spacing=(1, 1))
        for j, prop in enumerate(props):

            min_row, min_col, max_row, max_col = prop['bbox']
            mask = prop.image
            New_Red = Images[i][min_row:max_row, min_col:max_col, 0] * mask
            Normalized_Red = rescale_intensity(New_Red, in_range='image', out_range=(0, 1))  
            Normalized_Red = filters.gaussian(Normalized_Red, sigma=0.8)
            threshold = filters.threshold_otsu(Normalized_Red)
            # threshold = 0.3
            New_binary_image = np.clip((Normalized_Red > threshold).astype(int) + detect_outline(mask), 0, 1)
            
            #VISUALIZATION OF THE NUCLEI###
            # if path == paths[0]:
           
            #     fig, ax = plt.subplots(1, 3, figsize = (8,6))
    
            #     ax[0].imshow(mask, cmap = 'gray')
            #     ax[0].set_title('Mask', fontsize = 14)
            #     ax[1].imshow(Normalized_Red, cmap = 'gray')
            #     ax[1].set_title('Intensity image (Red channel)', fontsize = 14)
            #     ax[2].imshow(New_binary_image, cmap = 'gray')
            #     ax[2].set_title('Binary nucleus image', fontsize = 14)
            #     plt.tight_layout()
            #     plt.show()
            
            Red = (Normalized_Red* 255 / np.max(Normalized_Red)).astype(np.uint8)
            
            haralick_features = mh.features.haralick(Red).mean(0)

            def Category(region):
                category_value = 0 if path == paths[0] else 1
                return category_value
            def Intensity_std(region, intensities):
                return np.std(intensities[region], ddof=1)
            def Energy(region, intensities):
                return np.sum((intensities**2))
            def N_Objects(region):
                num_objects = label(region, return_num = True, connectivity=2)[1]
                return num_objects
            def Perimeter(region):
                return np.sum(detect_outline(mask))
            def N_Holes(region):
                return N_Objects(region) - Euler_number(region)  
            def Entropy(region):
                return shannon_entropy(region)
            def Euler_number(region):
                return euler_number(region, connectivity = 2)
            def Total_area_of_Holes(region):
                return np.sum((region^mask).flatten())
            def Contrast(region):
                return haralick_features[1]
            def Correlation(region):
                return haralick_features[2]
            def Homogeinity(region):
                return haralick_features[6]
            def ASM(region):
                return haralick_features[0]
            def Diff_Var(region):
                return haralick_features[9]
            def Diff_Entropy(region):
                return haralick_features[10]
            def IMC(region):
                return haralick_features[11]
            def Autocorrelation(region):
                return haralick_features[12]

            table = regionprops(New_binary_image, Normalized_Red, extra_properties=(Category, Intensity_std, Energy, N_Objects, Perimeter, 
                                                                                    N_Holes, Entropy, Euler_number,Total_area_of_Holes, Contrast, 
                                                                                    Correlation, Homogeinity, ASM, Diff_Var, Diff_Entropy, IMC, Autocorrelation))
            t = table[0]
            row = {
                'area': t.area,
                'area_bbox': t.bbox_area,
                'area_filled': t.filled_area,
                'area_convex': t.convex_area,
                'axis_major_length': t.major_axis_length,
                'axis_minor_length': t.minor_axis_length,
                'eccentricity': t.eccentricity,
                'equivalent_diameter_area': t.equivalent_diameter,
                'feret_diameter_max': t.feret_diameter_max,
                'intensity_max': t.max_intensity,
                'mean_intensity': t.mean_intensity,
                'intensity_min': t.min_intensity,
                'solidity': t.solidity,
                'num_pixels': t.area,
                'orientation': t.orientation,
                'Category': t.Category,
                'Intensity_std': t.Intensity_std,
                'Energy': t.Energy,
                'N_Objects': t.N_Objects, 
                'Perimeter': t.Perimeter,
                'N_Holes': t.N_Holes,
                'Entropy': t.Entropy,
                'euler_number': t.Euler_number,
                'Total_area_of_Holes': t.Total_area_of_Holes, 
                'Contrast': t.Contrast,
                'Correlation': t.Correlation,
                'Homogeinity': t. Homogeinity,
                'ASM': t.ASM, 
                'Diff_Var': t.Diff_Var, 
                'Diff_Entropy': t.Diff_Entropy,
                'IMC': t.IMC, 
                'Autocorrelation': t.Autocorrelation
            }

            # Calculate area percentage
            row['area_percentage'] = 100 - (row['area_filled'] - row['area']) / row['area_filled']

            all_props.append(row)
            Nuclei.append(Normalized_Red)

df = pd.DataFrame(all_props)
df.to_csv('Segmented_Nuclei_train.csv', index=False)

creating new log file
2024-06-02 01:57:26,815 [INFO] WRITING LOG OUTPUT TO C:\Users\George\.cellpose\run.log
2024-06-02 01:57:26,816 [INFO] 
cellpose version: 	3.0.8 
platform:       	win32 
python version: 	3.11.9 
torch version:  	2.2.2+cu118
2024-06-02 01:57:27,090 [INFO] >> denoise_nuclei << model set to be used
2024-06-02 01:57:27,095 [INFO] ** TORCH CUDA version installed and working. **
2024-06-02 01:57:27,095 [INFO] >>>> using GPU
2024-06-02 01:57:27,281 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)
2024-06-02 01:57:27,282 [INFO] >> nuclei << model set to be used
2024-06-02 01:57:27,300 [INFO] ** TORCH CUDA version installed and working. **
2024-06-02 01:57:27,300 [INFO] >>>> using GPU
2024-06-02 01:57:27,398 [INFO] >>>> loading model C:\Users\George\.cellpose\models\nucleitorch_0
2024-06-02 01:57:27,534 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)
2024-06-02 01:58:01,783 [INFO] >> denoise_nuclei << mode

In [144]:
# N = 5
# ira = np.zeros((Nuclei[N].shape[0], Nuclei[N].shape[1], 3))
# ira[:, :, 0], ira[:, :, 1], ira[:, :, 2] = Nuclei[N], 0, 0
# imshow(ira)